In [4]:
from datetime import datetime, timedelta, timezone
import os
import time
from random import choice
import regex as re
import json
from urllib.parse import parse_qs
###
# Third party imports
import pandas as pd
import requests
from bs4 import BeautifulSoup
import hashlib
from pivottablejs import pivot_ui
import warnings
warnings.filterwarnings('ignore')

In [5]:
# !pip install pandas

In [6]:
def get_random_user_agent() -> str:
    """
    This function returns a user agent signature for a set of predefined user agents.
    The returned user agent is used while scraping which is passed as a header.
    The purpose of using random user agents is to mimic a browser to make sure that
    FixedFloat doesn't identify our scraping script at regular intervals as a bot.
    """

    user_agents = [
        "Mozilla/5.0 (iPad; CPU OS 13_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/87.0.4280.77 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 8.0.0; SM-G955U Build/R16NW) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Mobile Safari/537.36",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_1 like Mac OS X) AppleWebKit/603.1.30 (KHTML, like Gecko) Version/10.0 Mobile/14E304 Safari/602.1",
    ]
    rand_choice = choice([i for i in range(0, len(user_agents))])
    user_agent = user_agents[rand_choice]
    return user_agent

def match_schedule_details(soup, season):
    get_matchscorecard = soup.find_all("table", attrs={"id": "ctl00_ContentPlaceHolder1_GridView1"})
    all_matches_schedule = pd.read_html(str(get_matchscorecard))[0][['Team','Opposition','Match Date','Ground']]
    get_matchscorecard = get_matchscorecard[0].find_all("a", attrs={"href": re.compile('.*Match*')})
    get_matchscorecard = [y.get("href") for y in get_matchscorecard]

    all_matches_schedule['matchscorecard_link'] = get_matchscorecard
    all_matches_schedule = all_matches_schedule.rename(columns = {'Match Date' : 'Match_Date'})
    all_matches_schedule['Match_Day'] = all_matches_schedule['Match_Date'].apply(lambda x : x.split()[0])
    all_matches_schedule['Match_Date'] = all_matches_schedule['Match_Date'].apply(lambda x : x.split()[1])
    all_matches_schedule['Match_Date'] = pd.to_datetime(all_matches_schedule['Match_Date'], format='%m/%d/%Y', errors='ignore').dt.strftime('%Y-%m-%d')
    # d = all_matches_schedule['Match_Date']
    all_matches_schedule['Ground'] = all_matches_schedule['Ground'].apply(lambda x : x.split(' (')[0])
    all_matches_schedule['season'] = season
    return all_matches_schedule

In [12]:
arcl_url = "https://arcl.org/Pages/UI/"

league_ids = [{'t_id': 5914, 'id' : 5, 's_id' : 59}, {'t_id': 5666, 'id' : 4, 's_id' : 57}, {'t_id': 5454, 'id' : 5, 's_id' : 56}, {'t_id': 5293, 'id' : 5, 's_id' : 50}, {'t_id': 4899, 'id' : 6, 's_id' : 47}, {'t_id': 4601, 'id' : 6, 's_id' : 46}, ]
batting_dfs_list = []
bowling_dfs_list = []
matches_dfs_list = []

for l in league_ids:
    team_stats_url = f"TeamStats.aspx?team_id={l['t_id']}&league_id={l['id']}&season_id={l['s_id']}"
    print(l)
    user_agent = (
        get_random_user_agent()
    )  # The purpose of using random user agents is to mimic a browser to make sure that FF doesn't identify our scraping at regular intervals as a bot
    arcl_session = requests.Session()
    req = arcl_session.get(
        arcl_url + team_stats_url, headers={"User-Agent": user_agent}, timeout=3
    )
    soup = BeautifulSoup(req.text, "html.parser")
    season = soup.find_all("p", attrs={"class": "centertitle"})[0].text
    print(season)
    
    _tmp_batting_df = pd.read_html(str(soup.find_all("table", attrs={"id": "ctl00_ContentPlaceHolder1_GridView2"})[0]))[0]
    _tmp_batting_df['season'] = season
    batting_dfs_list.append(_tmp_batting_df)
    
    _tmp_bowling_df = pd.read_html(str(soup.find_all("table", attrs={"id": "ctl00_ContentPlaceHolder1_GridView3"})[0]))[0]
    _tmp_bowling_df['season'] = season
    bowling_dfs_list.append(_tmp_bowling_df)

    _tmp_match_schedule_df = match_schedule_details(soup, season)
    matches_dfs_list.append(_tmp_match_schedule_df)

{'t_id': 5914, 'id': 5, 's_id': 59}
Spring 2023 - Div C
{'t_id': 5666, 'id': 4, 's_id': 57}
Summer 2022 - Div B
{'t_id': 5454, 'id': 5, 's_id': 56}
Spring 2022 - Div C
{'t_id': 5293, 'id': 5, 's_id': 50}
Summer 2021 - Div C
{'t_id': 4899, 'id': 6, 's_id': 47}
Summer 2019 - Div D
{'t_id': 4601, 'id': 6, 's_id': 46}
Spring 2019 - Div D


In [13]:
bowling_dfs_list[1]

,Player,Player_Id,Team,Innings,Overs,Maidens,Runs,Wickets,Average,Eco Rate,season
0,Phanindra Madduru,50314,CrickChamps,5,15.0,0,65,8,8,4.33,Summer 2022 - Div B
1,Madhu Paruchuri,62700,CrickChamps,2,6.0,0,25,6,4,4.17,Summer 2022 - Div B
2,Kowshik Goud,63316,CrickChamps,4,10.0,0,74,4,18,7.40,Summer 2022 - Div B
3,Avinash Naga,55618,CrickChamps,3,10.0,0,51,2,25,5.10,Summer 2022 - Div B
4,Deepak CC,56905,CrickChamps,5,9.0,0,55,2,27,6.11,Summer 2022 - Div B
5,Krishna Goli,64301,CrickChamps,3,6.0,0,51,2,25,8.50,Summer 2022 - Div B
6,Anirudh Tadakamalla,64302,CrickChamps,2,4.0,0,15,2,7,3.75,Summer 2022 - Div B
7,Yash Patil,64428,CrickChamps,1,3.0,0,18,2,9,6.00,Summer 2022 - Div B
8,Aviraj Sawant,64427,CrickChamps,3,12.0,0,55,1,55,4.58,Summer 2022 - Div B
9,Dheeraj Jonnala,53098,CrickChamps,1,3.0,0,28,1,28,9.33,Summer 2022 - Div B


In [14]:
df_team_players_batting = pd.concat(batting_dfs_list).reset_index(drop = True)
df_team_players_batting

,Player,Player_Id,Team,Innings,Runs,Balls,Fours,Sixs,Strike Rate,season
0,Deepak CC,56905,CrickChamps,2,67,71,3,0,94.37,Spring 2023 - Div C
1,Ashwin Muthu Kumar,57071,CrickChamps,2,59,53,0,2,111.32,Spring 2023 - Div C
2,Dheeraj Jonnala,53098,CrickChamps,2,28,37,1,0,75.68,Spring 2023 - Div C
3,Venkatesh Srinivasan,55617,CrickChamps,2,4,10,0,0,40.00,Spring 2023 - Div C
4,Madhu Paruchuri,62700,CrickChamps,1,3,3,0,0,100.00,Spring 2023 - Div C
...,...,...,...,...,...,...,...,...,...,...
76,Suresh Poliah,55616,CrickChamps,7,21,21,0,0,100.00,Spring 2019 - Div D
77,Sudheer CC,57073,CrickChamps,1,10,10,1,0,100.00,Spring 2019 - Div D
78,Siva Paturi,53097,CrickChamps,1,10,0,0,0,0.00,Spring 2019 - Div D
79,Pavan CC,54469,CrickChamps,3,7,11,0,0,63.64,Spring 2019 - Div D


In [15]:
df_team_players_bowling = pd.concat(bowling_dfs_list).reset_index(drop = True)
df_team_players_bowling

,Player,Player_Id,Team,Innings,Overs,Maidens,Runs,Wickets,Average,Eco Rate,season
0,Phanindra Madduru,50314,CrickChamps,1,4.0,0,17,2,8,4.25,Spring 2023 - Div C
1,Suresh Poliah,55616,CrickChamps,1,3.0,0,20,2,10,6.67,Spring 2023 - Div C
2,Krishna Ykc,64775,CrickChamps,1,2.0,0,19,1,19,9.50,Spring 2023 - Div C
3,Avinash Naga,55618,CrickChamps,1,4.0,0,16,1,16,4.00,Spring 2023 - Div C
4,Deepak CC,56905,CrickChamps,2,5.1,0,30,1,30,5.88,Spring 2023 - Div C
5,Ashwin Muthu Kumar,57071,CrickChamps,2,8.0,0,44,0,0,5.50,Spring 2023 - Div C
6,Madhu Paruchuri,62700,CrickChamps,1,3.0,0,14,0,0,4.67,Spring 2023 - Div C
7,Anirudh Tadakamalla,64302,CrickChamps,1,1.0,0,7,0,0,7.00,Spring 2023 - Div C
8,Venkatesh Srinivasan,55617,CrickChamps,1,1.0,0,11,0,0,11.00,Spring 2023 - Div C
9,Phanindra Madduru,50314,CrickChamps,5,15.0,0,65,8,8,4.33,Summer 2022 - Div B


In [16]:
all_matches_schedule = pd.concat(matches_dfs_list).reset_index(drop = True)
all_matches_schedule

,Team,Opposition,Match_Date,Ground,matchscorecard_link,Match_Day,season
0,CrickChamps,Seattle Blues,2023-03-25,Meadowdale Multipurpose Field 1,MatchScorecard.aspx?match_id=20659&league_id=5...,Saturday,Spring 2023 - Div C
1,CrickChamps,Torpedoes,2023-04-02,Ron Regis,MatchScorecard.aspx?match_id=20744&league_id=5...,Sunday,Spring 2023 - Div C
2,CrickChamps,Pirates of Seattle,2023-04-09,North Robinswood Park,MatchScorecard.aspx?match_id=20800&league_id=5...,Sunday,Spring 2023 - Div C
3,CrickChamps,Super Fit Challengers,2023-04-15,Ron Regis,MatchScorecard.aspx?match_id=20832&league_id=5...,Saturday,Spring 2023 - Div C
4,CrickChamps,Bouncers,2023-05-21,North SeaTac Park,MatchScorecard.aspx?match_id=21009&league_id=5...,Sunday,Spring 2023 - Div C
5,CrickChamps,Destroyers,2023-06-11,North SeaTac Park,MatchScorecard.aspx?match_id=21097&league_id=5...,Sunday,Spring 2023 - Div C
6,CrickChamps,Sidewinders Reloaded,2023-06-24,Redmond High School Stadium,MatchScorecard.aspx?match_id=21188&league_id=5...,Saturday,Spring 2023 - Div C
7,CrickChamps,Great Eight,2022-07-30,Redmond Ridge Park -- Soccer Field,MatchScorecard.aspx?match_id=20004&league_id=4...,Saturday,Summer 2022 - Div B
8,CrickChamps,Kachraboyzsecond,2022-08-07,North SeaTac Park,MatchScorecard.aspx?match_id=20079&league_id=4...,Sunday,Summer 2022 - Div B
9,CrickChamps,Back Breakers,2022-08-20,Marymoor Park Soccer #05,MatchScorecard.aspx?match_id=20153&league_id=4...,Saturday,Summer 2022 - Div B


In [17]:
# df_team_players_batting = pd.read_html(str(soup.find_all("table", attrs={"id": "ctl00_ContentPlaceHolder1_GridView2"})[0]))[0]
# df_team_players_batting

In [18]:
# df_team_players_bowling = pd.read_html(str(soup.find_all("table", attrs={"id": "ctl00_ContentPlaceHolder1_GridView3"})[0]))[0]
# df_team_players_bowling

In [19]:
# get_matchscorecard = soup.find_all("table", attrs={"id": "ctl00_ContentPlaceHolder1_GridView1"})
# all_matches_schedule = pd.read_html(str(get_matchscorecard))[0][['Team','Opposition','Match Date','Ground']]
# get_matchscorecard = get_matchscorecard[0].find_all("a", attrs={"href": re.compile('.*Match*')})
# get_matchscorecard = [y.get("href") for y in get_matchscorecard]

# all_matches_schedule['matchscorecard_link'] = get_matchscorecard
# all_matches_schedule = all_matches_schedule.rename(columns = {'Match Date' : 'Match_Date'})
# all_matches_schedule['Match_Day'] = all_matches_schedule['Match_Date'].apply(lambda x : x.split()[0])
# all_matches_schedule['Match_Date'] = all_matches_schedule['Match_Date'].apply(lambda x : x.split()[1])
# all_matches_schedule['Match_Date'] = pd.to_datetime(all_matches_schedule['Match_Date'], format='%m/%d/%Y', errors='ignore').dt.strftime('%Y-%m-%d')
# # d = all_matches_schedule['Match_Date']
# all_matches_schedule['Ground'] = all_matches_schedule['Ground'].apply(lambda x : x.split(' (')[0])
# all_matches_schedule

In [44]:
all_matches_schedule.dtypes

Team                   object
Opposition             object
Match_Date             object
Ground                 object
matchscorecard_link    object
Match_Day              object
season                 object
dtype: object

In [17]:
all_matches_schedule

,Team,Opposition,Match_Date,Ground,matchscorecard_link,Match_Day
0,CrickChamps,Seattle Blues,2023-03-25,Meadowdale Multipurpose Field 1,MatchScorecard.aspx?match_id=20659&league_id=5...,Saturday
1,CrickChamps,Torpedoes,2023-04-02,Ron Regis,MatchScorecard.aspx?match_id=20744&league_id=5...,Sunday
2,CrickChamps,Pirates of Seattle,2023-04-09,North Robinswood Park,MatchScorecard.aspx?match_id=20800&league_id=5...,Sunday
3,CrickChamps,Super Fit Challengers,2023-04-15,Ron Regis,MatchScorecard.aspx?match_id=20832&league_id=5...,Saturday
4,CrickChamps,Bouncers,2023-05-21,North SeaTac Park,MatchScorecard.aspx?match_id=21009&league_id=5...,Sunday
5,CrickChamps,Destroyers,2023-06-11,North SeaTac Park,MatchScorecard.aspx?match_id=21097&league_id=5...,Sunday
6,CrickChamps,Sidewinders Reloaded,2023-06-24,Redmond High School Stadium,MatchScorecard.aspx?match_id=21188&league_id=5...,Saturday


In [22]:
all_matches_schedule_df_json = eval(all_matches_schedule.to_json(orient='records'))
# all_matches_schedule_df_json[0:3], len(all_matches_schedule_df_json)

([{'Team': 'CrickChamps',
   'Opposition': 'Seattle Blues',
   'Match_Date': '2023-03-25',
   'Ground': 'Meadowdale Multipurpose Field 1',
   'matchscorecard_link': 'MatchScorecard.aspx?match_id=20659&league_id=5&season_id=59',
   'Match_Day': 'Saturday',
   'season': 'Spring 2023 - Div C'},
  {'Team': 'CrickChamps',
   'Opposition': 'Torpedoes',
   'Match_Date': '2023-04-02',
   'Ground': 'Ron Regis',
   'matchscorecard_link': 'MatchScorecard.aspx?match_id=20744&league_id=5&season_id=59',
   'Match_Day': 'Sunday',
   'season': 'Spring 2023 - Div C'},
  {'Team': 'CrickChamps',
   'Opposition': 'Pirates of Seattle',
   'Match_Date': '2023-04-09',
   'Ground': 'North Robinswood Park',
   'matchscorecard_link': 'MatchScorecard.aspx?match_id=20800&league_id=5&season_id=59',
   'Match_Day': 'Sunday',
   'season': 'Spring 2023 - Div C'}],
 46)

In [1]:
team_name = all_matches_schedule_df_json[0]['Team']

NameError: name 'all_matches_schedule_df_json' is not defined

In [23]:
df_player_batting_scores = pd.DataFrame()
df_player_bowling_scores = pd.DataFrame()

for match_payload in all_matches_schedule_df_json:
#     # print(arcl_url + matchresult_url)
    matchresult_url = match_payload['matchscorecard_link']
    match_date = match_payload['Match_Date']
    opposition = match_payload['Opposition']
    ground = match_payload['Ground']
    matchresult_html = pd.read_html(arcl_url + matchresult_url)
    match_data_json = json.dumps(parse_qs(matchresult_url.split('?')[1]))
    match_id = json.loads(match_data_json)['match_id'][0]
    league_id = json.loads(match_data_json)['league_id'][0]
    season_id = json.loads(match_data_json)['season_id'][0]
    season_name = match_payload['season']

    for each_batting_scores_table_id in [1,3]:
        try:
            tmp_df_player_batting_scores_per_match = matchresult_html[each_batting_scores_table_id][0:-3]
            tmp_df_player_batting_scores_per_match['match_id'] = match_id
            tmp_df_player_batting_scores_per_match['league_id'] = league_id
            tmp_df_player_batting_scores_per_match['season_id'] = season_id
            tmp_df_player_batting_scores_per_match['season'] = season_name
            tmp_df_player_batting_scores_per_match['match_date'] = match_date
            tmp_df_player_batting_scores_per_match['opposition'] = opposition
            tmp_df_player_batting_scores_per_match['ground'] = ground

            df_player_batting_scores = pd.concat([df_player_batting_scores,tmp_df_player_batting_scores_per_match])
        except Exception as e:
            print(e)
            
    for each_bowling_scores_table_id in [2,4]:
        try:
            tmp_df_player_bowling_scores_per_match = matchresult_html[each_bowling_scores_table_id]#[0:-2]
            tmp_df_player_bowling_scores_per_match['match_id'] = match_id
            tmp_df_player_bowling_scores_per_match['league_id'] = league_id
            tmp_df_player_bowling_scores_per_match['season_id'] = season_id
            tmp_df_player_bowling_scores_per_match['season'] = season_name
            tmp_df_player_bowling_scores_per_match['match_date'] = match_date
            tmp_df_player_bowling_scores_per_match['opposition'] = opposition
            tmp_df_player_bowling_scores_per_match['ground'] = ground

            df_player_bowling_scores = pd.concat([df_player_bowling_scores,tmp_df_player_bowling_scores_per_match])
        except Exception as e:
            print(e)
        time.sleep(1)

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [24]:
df_player_batting_scores = df_player_batting_scores.fillna(0)
df_player_batting_scores.head()

,Batter,How_out,Fielder,Bowler,Sixs,Fours,Runs,Balls,match_id,league_id,season_id,season,match_date,opposition,ground
0,Deepak CC,caught,Surya Wolfpack,Manjunath B,0.0,1.0,29,34.0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
1,Dheeraj Jonnala,bowled,0,Raju Siruvuri,0.0,1.0,11,18.0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
2,Rajiv Marothu,bowled,0,Raju Siruvuri,0.0,0.0,0,3.0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
3,Ashwin Muthu Kumar,caught,Manjunath B,Manjunath B,0.0,0.0,20,22.0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
4,Madhu Paruchuri,bowled,0,Nikhil Baireddy,0.0,0.0,3,3.0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1


In [26]:
# df_player_bowling_scores[(df_player_bowling_scores.Bowler == 'Total') & (df_player_bowling_scores.ground == 'Ron Regis')]

In [27]:
df_player_bowling_scores = df_player_bowling_scores.fillna(0)
df_player_bowling_scores.head()

,Bowler,Overs,Maiden,No_Balls,Wide,Runs,Wicket,match_id,league_id,season_id,season,match_date,opposition,ground
0,Vivek Singh,4.0,0.0,0.0,1.0,16,0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
1,Manjunath B,4.0,0.0,0.0,1.0,19,2,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
2,Raju Siruvuri,4.0,0.0,0.0,4.0,20,3,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
3,Karamveer Soni,2.0,0.0,0.0,2.0,14,0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
4,Nikhil Baireddy,2.0,0.0,0.0,0.0,9,1,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1


In [37]:
# df_player_batting_scores

In [33]:
final_df_batting = df_team_players_batting[['Player','Player_Id']].drop_duplicates().merge(df_player_batting_scores, left_on = 'Player', right_on = 'Batter', how = 'inner')
final_df_batting[['Sixs','Fours']] = final_df_batting[['Sixs','Fours']].fillna(0)
final_df_batting.head()

,Player,Player_Id,Batter,How_out,Fielder,Bowler,Sixs,Fours,Runs,Balls,match_id,league_id,season_id,season,match_date,opposition,ground
0,Deepak CC,56905,Deepak CC,caught,Surya Wolfpack,Manjunath B,0.0,1.0,29,34.0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
1,Deepak CC,56905,Deepak CC,bowled,0,Dhaval Shah,0.0,2.0,38,37.0,20744,5,59,Spring 2023 - Div C,2023-04-02,Torpedoes,Ron Regis
2,Deepak CC,56905,Deepak CC,bowled,0,Jatinder Singh Dhillon,0.0,0.0,4,0.0,20004,4,57,Summer 2022 - Div B,2022-07-30,Great Eight,Redmond Ridge Park -- Soccer Field
3,Deepak CC,56905,Deepak CC,caught,Ravi Samatham,Dewashish Chandramauli,5.0,4.0,65,34.0,20079,4,57,Summer 2022 - Div B,2022-08-07,Kachraboyzsecond,North SeaTac Park
4,Deepak CC,56905,Deepak CC,caught,Ranjit Unnikrishnan,Vishnu Vijayagopal,1.0,2.0,34,46.0,20227,4,57,Summer 2022 - Div B,2022-08-28,Pirates Of Seattle,Marymoor Park Soccer #05


In [34]:
final_df_batting[final_df_batting.match_id == '20659']

,Player,Player_Id,Batter,How_out,Fielder,Bowler,Sixs,Fours,Runs,Balls,match_id,league_id,season_id,season,match_date,opposition,ground
0,Deepak CC,56905,Deepak CC,caught,Surya Wolfpack,Manjunath B,0.0,1.0,29,34.0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
20,Ashwin Muthu Kumar,57071,Ashwin Muthu Kumar,caught,Manjunath B,Manjunath B,0.0,0.0,20,22.0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
36,Dheeraj Jonnala,53098,Dheeraj Jonnala,bowled,0,Raju Siruvuri,0.0,1.0,11,18.0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
52,Venkatesh Srinivasan,55617,Venkatesh Srinivasan,runout,Surya Wolfpack,Raju Siruvuri,0.0,0.0,4,10.0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
67,Madhu Paruchuri,62700,Madhu Paruchuri,bowled,0,Nikhil Baireddy,0.0,0.0,3,3.0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
82,Rajiv Marothu,57070,Rajiv Marothu,bowled,0,Raju Siruvuri,0.0,0.0,0,3.0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
111,Krishna Ykc,64775,Krishna Ykc,caught,Surya Wolfpack,Raju Siruvuri,0.0,0.0,2,4.0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
112,Avinash Naga,55618,Avinash Naga,not out,0,0,0.0,0.0,1,2.0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1


In [28]:
final_df_bowling = df_team_players_bowling[['Player','Player_Id']].drop_duplicates().merge(df_player_bowling_scores, left_on = 'Player', right_on = 'Bowler', how = 'inner')
# final_df[['Sixs','Fours']] = final_df[['Sixs','Fours']].fillna(0)
final_df_bowling.head()

,Player,Player_Id,Bowler,Overs,Maiden,No_Balls,Wide,Runs,Wicket,match_id,league_id,season_id,season,match_date,opposition,ground
0,Phanindra Madduru,50314,Phanindra Madduru,4.0,0.0,0.0,1.0,17,2,20744,5,59,Spring 2023 - Div C,2023-04-02,Torpedoes,Ron Regis
1,Phanindra Madduru,50314,Phanindra Madduru,2.0,0.0,0.0,0.0,8,1,20004,4,57,Summer 2022 - Div B,2022-07-30,Great Eight,Redmond Ridge Park -- Soccer Field
2,Phanindra Madduru,50314,Phanindra Madduru,2.0,0.0,0.0,1.0,11,1,20079,4,57,Summer 2022 - Div B,2022-08-07,Kachraboyzsecond,North SeaTac Park
3,Phanindra Madduru,50314,Phanindra Madduru,4.0,0.0,0.0,3.0,28,1,20153,4,57,Summer 2022 - Div B,2022-08-20,Back Breakers,Marymoor Park Soccer #05
4,Phanindra Madduru,50314,Phanindra Madduru,4.0,0.0,0.0,2.0,13,2,20227,4,57,Summer 2022 - Div B,2022-08-28,Pirates Of Seattle,Marymoor Park Soccer #05


In [29]:
df_player_bowling_scores[df_player_bowling_scores.match_id == '20659']

,Bowler,Overs,Maiden,No_Balls,Wide,Runs,Wicket,match_id,league_id,season_id,season,match_date,opposition,ground
0,Vivek Singh,4.0,0.0,0.0,1.0,16,0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
1,Manjunath B,4.0,0.0,0.0,1.0,19,2,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
2,Raju Siruvuri,4.0,0.0,0.0,4.0,20,3,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
3,Karamveer Soni,2.0,0.0,0.0,2.0,14,0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
4,Nikhil Baireddy,2.0,0.0,0.0,0.0,9,1,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
5,Byes,0.0,0.0,0.0,0.0,1,0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
6,Total,0.0,0.0,0.0,0.0,79,0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
0,Ashwin Muthu Kumar,4.0,0.0,0.0,0.0,20,0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
1,Krishna Ykc,2.0,0.0,1.0,8.0,19,1,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
2,Avinash Naga,4.0,0.0,0.0,0.0,16,1,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1


In [30]:
# import seaborn as sns
# sns.histplot(data=final_df, x="Player", y ="Runs", hue="Player", multiple="dodge", shrink=.8)
# sns.histplot(data=final_df, x="Player", y ="Runs", alpha = .7)

# final_df.plot(x = "match_id", y=["Runs", "Balls"], kind="bar", figsize=(20, 8) )

final_df_bowling_for_how_wickets_taken = df_team_players_batting[['Player','Player_Id']].drop_duplicates().merge(df_player_batting_scores, left_on = 'Player', right_on = 'Bowler', how = 'inner')
final_df_bowling_for_how_wickets_taken

,Player,Player_Id,Batter,How_out,Fielder,Bowler,Sixs,Fours,Runs,Balls,match_id,league_id,season_id,season,match_date,opposition,ground
0,Deepak CC,56905,Vivek Singh,caught,Rajiv Marothu,Deepak CC,0.0,1.0,16,19.0,20659,5,59,Spring 2023 - Div C,2023-03-25,Seattle Blues,Meadowdale Multipurpose Field 1
1,Deepak CC,56905,Arun C Pentapalli,caught,Suresh Poliah,Deepak CC,1.0,2.0,31,25.0,20302,4,57,Summer 2022 - Div B,2022-09-11,Stingrays,Ron Regis
2,Deepak CC,56905,Vernon Dsouza,caught,Venkatesh Srinivasan,Deepak CC,0.0,3.0,16,19.0,20571,4,57,Summer 2022 - Div B,2022-10-15,Seattle United - Supercharged,Hidden Valley Park Field 1
3,Deepak CC,56905,Ashwini Singh,caught,Ashwin Muthu Kumar,Deepak CC,0.0,0.0,2,9.0,17731,6,47,Summer 2019 - Div D,2019-07-13,Eight Musketers,Redmond High School Stadium
4,Deepak CC,56905,Binu Blazers,bowled,0,Deepak CC,0.0,0.0,4,0.0,17955,6,47,Summer 2019 - Div D,2019-08-03,FatehCC-Blazers,Redmond Ridge Park -- Soccer Field
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,Srinivas Potluri,57069,Hari Pirates of Seattle 5026,caught,Suresh Poliah,Srinivas Potluri,0.0,0.0,5,6.0,16687,6,46,Spring 2019 - Div D,2019-06-02,Pirates of Seattle,North Robinswood Park
208,Srinivas Potluri,57069,Santosh Pirates of Seattle 12305,caught,Srinivas Potluri,Srinivas Potluri,1.0,0.0,13,14.0,16687,6,46,Spring 2019 - Div D,2019-06-02,Pirates of Seattle,North Robinswood Park
209,Srinivas Potluri,57069,Bala Kannan,caught,Suresh Poliah,Srinivas Potluri,0.0,0.0,1,2.0,16770,6,46,Spring 2019 - Div D,2019-06-15,Gilly,Eastlake High School Stadium
210,Sudheer CC,57073,Anil Ramchandani,stumped,Avinash Naga,Sudheer CC,0.0,0.0,19,0.0,17955,6,47,Summer 2019 - Div D,2019-08-03,FatehCC-Blazers,Redmond Ridge Park -- Soccer Field


In [37]:
TEMPLATE = u"""
<!DOCTYPE html>
<html>
    <head>
        <meta charset="UTF-8">
        <title>PivotTable.js</title>
        <!-- external libs from cdnjs -->
        <link rel="stylesheet" type="text/css" href="https://cdnjs.cloudflare.com/ajax/libs/c3/0.4.11/c3.min.css">
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js"></script>
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/c3/0.4.11/c3.min.js"></script>
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/jquery/1.11.2/jquery.min.js"></script>
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/jqueryui/1.11.4/jquery-ui.min.js"></script>
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/jquery-csv/0.71/jquery.csv-0.71.min.js"></script>
        <link rel="stylesheet" type="text/css" href="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/pivot.min.css">
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/pivot.min.js"></script>
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/d3_renderers.min.js"></script>
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/c3_renderers.min.js"></script>
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/export_renderers.min.js"></script>
        <style>
            body {font-family: Verdana;}
            .node {
              border: solid 1px white;
              font: 10px sans-serif;
              line-height: 12px;
              overflow: hidden;
              position: absolute;
              text-indent: 2px;
            }
            .c3-line, .c3-focused {stroke-width: 3px !important;}
            .c3-bar {stroke: white !important; stroke-width: 1;}
            .c3 text { font-size: 12px; color: grey;}
            .tick line {stroke: white;}
            .c3-axis path {stroke: grey;}
            .c3-circle { opacity: 1 !important; }
            .c3-xgrid-focus {visibility: hidden !important;}
        </style>
    </head>
    <body>
        <script type="text/javascript">
            $(function(){
                $("#output").pivotUI(
                    $.csv.toArrays($("#output").text())
                    , $.extend({
                        renderers: $.extend(
                            $.pivotUtilities.renderers,
                            $.pivotUtilities.c3_renderers,
                            $.pivotUtilities.d3_renderers,
                            $.pivotUtilities.export_renderers
                            ),
                        hiddenAttributes: [""]
                    } 
                    , {
                        onRefresh: function(config) {
                            var config_copy = JSON.parse(JSON.stringify(config));
                            //delete some values which are functions
                            delete config_copy["aggregators"];
                            delete config_copy["renderers"];
                            //delete some bulky default values
                            delete config_copy["rendererOptions"];
                            delete config_copy["localeStrings"];
                            $("#output2").text(JSON.stringify(config_copy, undefined, 2));
                        }
                    }
                    , %(kwargs)s
                    , %(json_kwargs)s)
                ).show();
             });
        </script>
        <div id="output" style="display: none;">%(csv)s</div>
        <textarea id="output2"
        style="float: left; width: 0px; height: 0px; margin: 0px; opacity:0;" readonly>
        </textarea>
        <button onclick="copyTextFunction()">Copy settings</button>
        <script>
        function copyTextFunction() {
                    var copyText = document.getElementById("output2");
                    copyText.select();
                    document.execCommand("copy");
                    }
        </script>
    </body>
</html>
"""


def pivot_cht_ui(df, name, url="", width="100%", height="500",json_kwargs='',  **kwargs):
    from IPython.display import HTML
    from IPython.display import IFrame
    import io
    print(name)
    outfile_path = name + '.html'
    with io.open(outfile_path, 'wt', encoding='utf8') as outfile:
        csv = df.to_csv(encoding='utf8')
        if hasattr(csv, 'decode'):
            csv = csv.decode('utf8')
        outfile.write(TEMPLATE % dict(csv=csv, kwargs=json.dumps(kwargs),json_kwargs=json_kwargs))
  
    return IFrame(src=url or outfile_path, width=width, height=height)(final_df_bowling_for_how_wickets_taken, rows=['match_date'], cols=['Player'], vals=['Runs'], aggregatorName='Sum', rendererName='Bar Chart', menuLimit=50, showUI = False, outfile_path="bowling_wickets_v2.html")

In [38]:
pivot_cht_ui(final_df_batting, name = 'final_df_batting', url="", width="100%", height="500",json_kwargs='')

final_df_batting


TypeError: 'IFrame' object is not callable

In [42]:
pivot_ui(final_df_batting, rows=['Player'], cols=['season_id','season'], vals=['Runs'], aggregatorName='Sum', rendererName='Row Heatmap', menuLimit=50, showUI = False, outfile_path="batting_runs_scored_v2.html")

In [39]:
pivot_ui(final_df_batting, rows=['Player'], cols=['How_out'], vals=['Runs'], aggregatorName='Count', rendererName='Row Heatmap', menuLimit=50, showUI = True, outfile_path="batting_how_out_v2.html")

In [41]:
pivot_ui(final_df_bowling, rows=['Player'], cols=['season_id','season'], vals=['Runs','Overs'], aggregatorName='Sum over Sum', rendererName='Row Heatmap', menuLimit=50, showUI = False, outfile_path="bowling_runs_given.html")

In [40]:
pivot_ui(final_df_bowling, rows=['match_date'], cols=['Player'], vals=['Wicket'], aggregatorName='Sum', rendererName='Bar Chart', menuLimit=50, showUI = False, outfile_path="bowling_wickets_taken_v2.html")

In [329]:
# %%javascript
# var e = document.createElement('script'); e.setAttribute('src', 'https://nytimes.github.io/svg-crowbar/svg-crowbar.js'); e.setAttribute('class', 'svg-crowbar'); document.body.appendChild(e);